In [1]:
!pip install selectolax==0.3.12
!pip install playwright==1.30.0
!playwright install chromium
!apt install chromium-chromedriver
!pip install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.9/539.9 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 3.0.3
    Uninstalling greenlet-3.0.3:
      Successfully uninstalled greenlet-3.0.3
141.7 Mb [] 0% 0.0s141.7 Mb [] 0% 24.3s141.7 Mb [] 0% 16.3s141.7 Mb [] 0% 9.0s141.7 Mb [] 1% 6.7s141.7 Mb [] 1% 7.4s141.7 Mb [] 1% 8.4s141.7 Mb [] 1% 9.4s141.7 Mb [] 1% 10.2s141.7 Mb [] 1% 9.5s141.7 Mb [] 2% 9.0s141.7 Mb [] 2% 8.0s141.7 Mb [] 2% 8.3s141.7 Mb [] 2% 8.8s141.7 Mb [] 2% 9.4s141.7 Mb [] 2% 9.7s141.7 Mb [] 3% 9.4s141.7 Mb [] 3% 8.2s141.7 Mb [] 4% 8.1s141.7 Mb [] 4% 8.5s141.7 Mb [] 4% 8.1s141.7 Mb [] 4% 7.9s141.7 Mb [] 5% 7.6s141.7 Mb [] 5% 7.3s141.7 Mb [] 6% 6.7s141.7 Mb [] 6% 6.9s141.7 Mb [] 6% 7.0s141.7 Mb [] 6% 6.8s141.7 Mb [] 7% 6.6s141.7 Mb [] 8% 6.0s141.7 Mb [] 8% 5.9s141.7 

In [2]:
import asyncio
from playwright.async_api import async_playwright
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import nest_asyncio
nest_asyncio.apply()

In [3]:
url = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"
header = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'
}

In [4]:
async def extract_full_body_html(url):
    TIMEOUT = 90000
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'  # Replace with your desired User-Agent
    }
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        # Set the User-Agent header
        await page.set_extra_http_headers(headers)

        await page.goto(url)

        # Wait until network is idle and the specific selector is loaded
        await page.wait_for_load_state('networkidle')
        await page.wait_for_selector(".sc-ed803445-0", timeout=TIMEOUT)


        # Extract the full HTML content of the body
        body_html = await page.inner_html('body')

        # Close the browser
        await browser.close()

        return body_html

In [5]:
async def main():
    full_body_html = await extract_full_body_html(url)
    return full_body_html

In [7]:
full_body_html = asyncio.run(main())

In [8]:
soup = bs(full_body_html,'html.parser')

In [9]:
movies = soup.select('li.ipc-metadata-list-summary-item')

In [10]:
def extract_movie_info(movie):
    title = movie.select('div a h3')[0].get_text(strip=True)
    span_tags = movie.select('span.sc-b189961a-8')
    span_texts = [span.text for span in span_tags]
    release_year = span_texts[0] if len(span_texts) > 0 else None
    movie_length = span_texts[1] if len(span_texts) > 1 else None
    category = span_texts[2] if len(span_texts) > 2 else None
    return {
       "title":title,
       "release_year":release_year,
       "movie_length":movie_length,
       "category":category
      }

In [11]:
extract_movie_info(movies[0])

{'title': 'Beverly Hills Cop: Axel F',
 'release_year': '2024',
 'movie_length': '1h 58m',
 'category': 'R'}

In [12]:
movies_info = [extract_movie_info(movie) for movie in movies]

In [13]:
movies_info

[{'title': 'Beverly Hills Cop: Axel F',
  'release_year': '2024',
  'movie_length': '1h 58m',
  'category': 'R'},
 {'title': 'A Family Affair',
  'release_year': '2024',
  'movie_length': '1h 51m',
  'category': 'PG-13'},
 {'title': 'A Quiet Place: Day One',
  'release_year': '2024',
  'movie_length': '1h 39m',
  'category': 'PG-13'},
 {'title': 'Kalki 2898 AD',
  'release_year': '2024',
  'movie_length': '3h',
  'category': None},
 {'title': 'Inside Out 2',
  'release_year': '2024',
  'movie_length': '1h 36m',
  'category': 'PG'},
 {'title': 'Horizon: An American Saga - Chapter 1',
  'release_year': '2024',
  'movie_length': '3h 1m',
  'category': 'R'},
 {'title': 'Furiosa: A Mad Max Saga',
  'release_year': '2024',
  'movie_length': '2h 28m',
  'category': 'R'},
 {'title': 'Longlegs',
  'release_year': '2024',
  'movie_length': '1h 41m',
  'category': 'R'},
 {'title': 'MaXXXine',
  'release_year': '2024',
  'movie_length': '1h 43m',
  'category': 'R'},
 {'title': 'Deadpool & Wolverin

In [14]:
import pandas as pd
df = pd.DataFrame(movies_info)
df.head()

,title,release_year,movie_length,category
0,Beverly Hills Cop: Axel F,2024,1h 58m,R
1,A Family Affair,2024,1h 51m,PG-13
2,A Quiet Place: Day One,2024,1h 39m,PG-13
3,Kalki 2898 AD,2024,3h,None
4,Inside Out 2,2024,1h 36m,PG


In [15]:
len(df)

100